In [1]:
from utils.df_handle import *

Default bq project:  spatial-vision-343005.biteam


In [2]:
sql = \
"""
SELECT BusinessScope FROM dbo.AR_Customer 
"""

In [10]:
df = get_ms_df(sql)

In [13]:
df.BusinessScope.replace('', None, inplace=True)
df.BusinessScope = df.BusinessScope.str.replace('05','1').str.replace('06','2').str.replace('07','3').str.replace('08','4').str.replace('09','5')

In [18]:
df.BusinessScope.head()

0     2,3
1     2,3
2     2,3
3     2,3
4    None
Name: BusinessScope, dtype: object

In [ ]:
import pendulum
from airflow import DAG
from airflow.operators.dummy_operator import DummyOperator
from airflow.operators.python_operator import PythonOperator

local_tz = pendulum.timezone("Asia/Bangkok")
name='MASTER_CUSTOMER'
prefix='DMS_'
path = f'/usr/local/airflow/plugins/{prefix}{name}/'

# datenow_min1 = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")

In [ ]:
dag_params = {
    'owner': 'airflow',
    "depends_on_past": False,
    'start_date': datetime(2022, 4, 14, tzinfo=local_tz),
    'email_on_failure': True,
    'email_on_retry': False,
    'email':['duyvq@merapgroup.com', 'vanquangduy10@gmail.com'],
    'do_xcom_push': False,
    'execution_timeout':timedelta(seconds=300)
    # 'retries': 3,
    # 'retry_delay': timedelta(minutes=10),
}

dag = DAG(prefix+name,
          catchup=False,
          default_args=dag_params,
          schedule_interval= '0 1 * * *',
          tags=[prefix+name, 'Daily', 'at0']
)

In [2]:
datenow_min1 = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")

sql = \
f"""
With #InvoiceCount as
(
SELECT * FROM
(
SELECT custInvoice.CustIDInvoice, custInvoice.CustID, Invoice.CustNameInvoice, Invoice.CountryID, Invoice.State, 
Invoice.DistrictID, Invoice.Ward, Invoice.ApartNumber, Invoice.StreetName, Invoice.TaxID, ROW_NUMBER()
OVER (PARTITION BY CustID ORDER BY (SELECT NULL)) AS RowNum
FROM    AR_Customer_InvoiceCustomer custInvoice WITH(NOLOCK) 
INNER JOIN AR_CustomerInvoice Invoice WITH(NOLOCK) ON Invoice.CustIDInvoice = custInvoice.CustIDInvoice  
where Active = 1
) t
WHERE t.RowNum=1
)
SELECT DISTINCT a.BranchID, 
BranchName= s.CpnyName,
a.CustId, 
RefCustID,
--arcic.CustIDInvoice,
--arci.CustNameInvoice,
CustName,
Address = CAST ((
                    ISNULL(a.Addr1, '') + CASE ISNULL(a.Addr2, '') WHEN '' THEN '' ELSE ', '+ a.Addr2 END +
                    CASE ISNULL(a.Ward, '') WHEN '' THEN '' ELSE ', ' + ISNULL(w.Name, a.Ward) END +
                    CASE ISNULL(a.District, '') WHEN '' THEN '' ELSE ', '+ ISNULL(dt.Name, a.District) END +
                    CASE ISNULL(a.State, '') WHEN '' THEN '' ELSE ', '+ ISNULL(st.Descr, a.State) END + 
                    CASE ISNULL(a.Country, '') WHEN '' THEN '' ELSE ', '+ ISNULL(cou.Descr, a.Country) END
                    ) AS NVARCHAR(MAX)),
a.District as DistrictCode,
a.State as StateCode,
a.Ward as WardCode,
a.Country as CountryCode,
a.Territory as TerritoryCode,
a.SalesSystem,
DistrictDescr = dt.Name,
StateDescr = st.Descr,
TerritoryDescr = t.Descr,
CustInvoice1.[CustIDInvoice],
    Addr1 =   
CASE ISNULL(ApartNumber, '') WHEN '' THEN '' ELSE ApartNumber END   +  
--CASE ISNULL(StreetName, '') WHEN '' THEN '' ELSE + IIF(ISNULL(ApartNumber, '') = '', '', ', ') + StreetName END +  
CASE ISNULL(CustInvoice1.Ward, '') WHEN '' THEN '' ELSE ', ' + ISNULL(war.Name, CustInvoice1.Ward) END +  
CASE ISNULL(CustInvoice1.DistrictID, '') WHEN '' THEN '' ELSE ', '+ ISNULL(dis.Name, CustInvoice1.DistrictID) + ', ' END +  
--CASE CustInvoice1.State WHEN '13' THEN ''  
--       WHEN '15' THEN ''  
--       WHEN '24' THEN ''  
--       WHEN '30' THEN ''  
--       WHEN '28' THEN N'Thành Phố' + ' '   
--       else N'Tỉnh' + ' ' end +  --20210505 trung ht bỏ ',' đằng trước tỉnh
CASE ISNULL(CustInvoice1.State, '') WHEN '' THEN '' ELSE ISNULL(sta.Descr, CustInvoice1.State) END +  
CASE ISNULL(CustInvoice1.CountryID, '') WHEN '' THEN '' ELSE ', '+ ISNULL(coun.Descr, CustInvoice1.CountryID) END  ,
TaxRegNbr = CustInvoice1.TaxID ,
a.Channel,
a.ShopType,
ShopTypeDescr = isnull(sh.Descr, ''),
a.Phone,
HCOTypeID = hco.HCOTypeName,
a.ClassId,
PaymentsForm = pay.Descr,
Terms = isnull(sterm.Descr, ''),
Active = case when a.[Status] = 'A' then N'Active' when a.[Status] = 'I' then N'Inactive' else 'Pending' end, --ISNULL(dbo.fr_Language(ap.LangStatus,@LangID), a.Status),
InActive = a.InActive,
AutoGenOrder = case when a.GenOrders ='KTHD' then N'Không tạo hóa đơn'
                when a.GenOrders ='MDHMHD' then N'Một đơn hàng một hóa đơn'
                when a.GenOrders ='THDHKM' then N'Tách hóa đơn hàng KM'
                WHEN a.GenOrders ='THDTTS' then N'Tách hóa đơn theo thuế suất'
                when a.GenOrders ='THDTSP' then N'Tách hóa đơn từng sản phẩm'
                end,
arcl.Lat,
arcl.Lng,
a.Crtd_Datetime,
a.LUpd_Datetime
FROM AR_Customer a WITH(NOLOCK)
INNER JOIN SYS_Company s WITH(NOLOCK) ON s.CpnyID = a.BranchID
LEFT JOIN #InvoiceCount  CustInvoice1 WITH(NOLOCK) ON CustInvoice1.CustID = a.CustId 
LEFT JOIN SI_Country cou WITH(NOLOCK) ON cou.CountryID = a.Country
LEFT JOIN SI_State st WITH(NOLOCK) ON st.State = a.State AND st.Country = a.Country
LEFT JOIN SI_District dt WITH(NOLOCK) ON a.District = dt.District AND a.Territory = dt.Territory AND dt.State = st.State AND dt.Country = a.Country
LEFT JOIN dbo.SI_Ward w WITH(NOLOCK) ON w.State = a.State AND w.Ward = a.Ward AND w.District = a.District
LEFT JOIN SI_Territory t WITH(NOLOCK) ON t.Territory = a.Territory
LEFT JOIN AR_ShopType sh WITH(NOLOCK) ON sh.Code = a.ShopType
LEFT JOIN SI_Terms sterm WITH(NOLOCK) ON sterm.TermsID = a.Terms
LEFT JOIN AR_MasterPayments pay WITH(NOLOCK) ON pay.Code = a.PaymentsForm
LEFT JOIN AR_HCOType hco WITH(NOLOCK) ON hco.HCOTypeID = a.HCOTypeID AND  hco.HCOID = a.HCOID
LEFT JOIN dbo.AR_HCO ah ON ah.HCOID=a.HCOID
LEFT JOIN SI_ApprovalFlowStatus ap  WITH(NOLOCK) ON ap.Status = a.Status AND ap.AppFolID = 'AR20400'
LEFT JOIN SI_Country as coun WITH(NOLOCK) on coun.CountryID = CustInvoice1.CountryID  
LEFT JOIN SI_State as sta WITH(NOLOCK) on  sta.Country = CustInvoice1.CountryID AND sta.State = CustInvoice1.State  
LEFT JOIN SI_District as dis WITH(NOLOCK) on  dis.District = CustInvoice1.DistrictID  
LEFT JOIN SI_Ward as war WITH(NOLOCK)  on  war.Country = CustInvoice1.CountryID AND war.State = CustInvoice1.State AND war.District = CustInvoice1.DistrictID AND war.Ward = CustInvoice1.Ward
LEFT JOIN AR_CustomerLocation as arcl WITH(NOLOCK)  on  arcl.CustID = a.CustID AND arcl.BranchID = a.BranchID
"""

In [3]:
df = get_ms_df(sql)

In [4]:
df['inserted_at'] = datetime.now()

In [6]:
df.columns = lower_col(df)

In [9]:
df.columns

Index(['branchid', 'branchname', 'custid', 'refcustid', 'custname', 'address',
       'districtcode', 'statecode', 'wardcode', 'countrycode', 'territorycode',
       'salessystem', 'districtdescr', 'statedescr', 'territorydescr',
       'custidinvoice', 'addr1', 'taxregnbr', 'channel', 'shoptype',
       'shoptypedescr', 'phone', 'hcotypeid', 'classid', 'paymentsform',
       'terms', 'active', 'inactive', 'autogenorder', 'lat', 'lng',
       'crtd_datetime', 'lupd_datetime', 'inserted_at'],
      dtype='object')

In [10]:
df.columns = lower_col(df)
df.statecode = df.statecode.astype('float')
df.districtcode = df.districtcode.astype('float')
df.wardcode = df.wardcode.astype('float')
df.countrycode = df.countrycode.astype('float')
df.territorycode = df.territorycode.astype('float')
df.salessystem = df.salessystem.astype('float')

In [11]:
# df.statecode

In [12]:
bq_values_insert(df, "d_master_khachhang", 2)

1it [00:20, 20.76s/it]


In [4]:
def update_customer():
    df = get_ms_df(sql)
    try:
        assert df.shape[0] >0
    except AssertionError:
        print("No customer changed")
    else:
        df.columns = cleancols(df)
        df.columns = lower_col(df)
        df['crtd_date'] = df.crtd_datetime.dt.normalize()
        df1 = df['crtd_date']
        # df1.dt.strftime('%Y-%m-%d')
        df1.drop_duplicates(inplace=True)
        tpl_dt = tuple(df1.dt.strftime('%Y-%m-%d').to_list())
        df1 = df['custid']
        df1.drop_duplicates(inplace=True)
        tpl_ct = tuple(df1.to_list())
        del_sql = \
        f"""
        DELETE FROM biteam.d_master_khachhang
        WHERE
        DATE(crtd_datetime) in {tpl_dt}
        AND custid in {tpl_ct}
        """
        execute_bq_query(del_sql)
        df['inserted_at'] = datetime.now()
        drop_cols(df, 'crtd_date')
        bq_values_insert(df, "d_master_khachhang", 2)

In [5]:
update_customer()

1it [00:16, 16.60s/it]


In [ ]:
dummy_start = DummyOperator(task_id="dummy_start", dag=dag)

update_customer = PythonOperator(task_id="update_customer", python_callable=update_customer, dag=dag)

dummy_start >> update_customer